In [86]:
#Import necessary library to analyst the data 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [87]:
#import email 
email = pd.read_csv('../data/email.csv')

In [88]:
email.head()

,id,date,user,pc,to,cc,bcc,from,size,attachments,content
0,{R3I7-S4TX96FG-8219JWFF},01/02/2010 07:11:45,LAP0338,PC-5758,Dean.Flynn.Hines@dtaa.com;Wade_Harrison@lockhe...,Nathaniel.Hunter.Heath@dtaa.com,NaN,Lynn.Adena.Pratt@dtaa.com,25830,0,middle f2 systems 4 july techniques powerful d...
1,{R0R9-E4GL59IK-2907OSWJ},01/02/2010 07:12:16,MOH0273,PC-6699,Odonnell-Gage@bellsouth.net,NaN,NaN,MOH68@optonline.net,29942,0,the breaking called allied reservations former...
2,{G2B2-A8XY58CP-2847ZJZL},01/02/2010 07:13:00,LAP0338,PC-5758,Penelope_Colon@netzero.com,NaN,NaN,Lynn_A_Pratt@earthlink.net,28780,0,slowly this uncinus winter beneath addition ex...
3,{A3A9-F4TH89AA-8318GFGK},01/02/2010 07:13:17,LAP0338,PC-5758,Judith_Hayden@comcast.net,NaN,NaN,Lynn_A_Pratt@earthlink.net,21907,0,400 other difficult land cirrocumulus powered ...
4,{E8B7-C8FZ88UF-2946RUQQ},01/02/2010 07:13:28,MOH0273,PC-6699,Bond-Raymond@verizon.net;Alea_Ferrell@msn.com;...,NaN,Odonnell-Gage@bellsouth.net,MOH68@optonline.net,17319,0,this kmh october holliswood number advised unu...


In [89]:
email.columns

Index(['id', 'date', 'user', 'pc', 'to', 'cc', 'bcc', 'from', 'size',
       'attachments', 'content'],
      dtype='object')

In [ ]:
#Convert the data format 
email['date'] = pd.to_datetime(email['date'], format='%m/%d/%Y %H:%M:%S')

In [ ]:
# Frequency of emails by user
user_frequency = email['user'].value_counts()

# Number of attachments analysis (if the number of attachments is higher it should notice risk
attachment_analysis = email['attachments'].value_counts()

In [ ]:
user_frequency

In [ ]:
email_ex = email.query("user == 'ABC0174'")
email_ex

In [ ]:
#@dtaa.com is company email 
def contains_dtaa(row):
    relevant_columns = ['to', 'cc', 'bcc']
    for col in relevant_columns:
        if pd.notnull(row[col]):
            # Split the column value by semicolon and check each email
            emails = row[col].split(';')
            if any('@dtaa.com' in email.strip() for email in emails):
                return True
    return False

# Create the 'not_dtaa' column
email['not_dtaa'] = ~email.apply(contains_dtaa, axis=1)

In [ ]:
#Check values fo email
email['not_dtaa'].value_counts()

In [ ]:
#remove @dtaa.com because 
email_filtered = email[email['not_dtaa'] == True].reset_index(drop=True)

In [ ]:
email_filtered.query("user == 'RAR0725'")

In [ ]:
user_frequency = email_filtered['user'].value_counts()

In [ ]:
user_frequency

In [ ]:
#check the remaining TP users(1)
email_filtered['user'].to_csv('../data/user_frequency.csv', index=False)
#70 remaining TP cases


In [ ]:
user_frequency.mean()

In [ ]:
sizing = email_filtered['size'].value_counts()

In [ ]:
sizing

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(user_frequency, kde=True, bins=30, color='blue')
plt.title('User Frequency Distribution', fontsize=16)
plt.xlabel('Email Count per User', fontsize=14)
plt.ylabel('Frequency', fontsize=14);

In [ ]:
#find the top users to 
top_users = email_filtered['user'].value_counts().head(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_users.index, y=top_users.values)
plt.title('Top 10 Users by Email Activity')
plt.xlabel('User')
plt.ylabel('Email Count')
plt.show()

In [ ]:
# impute Missing values in 'cc' and 'bcc'
email_filtered['cc'] = email_filtered['cc'].fillna('')
email_filtered['bcc'] = email_filtered['bcc'].fillna('')

#summary number of recipient from to cc, bcc
email_filtered['num_recipients'] = email_filtered['to'].str.count(';') + email_filtered['cc'].str.count(';') + email_filtered['bcc'].str.count(';') + 1
email_filtered['hour'] = email_filtered['date'].dt.hour
email_filtered['day_of_week'] = email_filtered['date'].dt.dayofweek 

# Define working hours and working days
working_hours_start = 9 # 9 AM
working_hours_end = 17  # 5 PM
working_days = [0, 1, 2, 3, 4]  # Monday=0, Sunday=6

#is working_time columns 
email_filtered['is_working_time'] = (
    (email_filtered['hour'] >= working_hours_start) & 
    (email_filtered['hour'] < working_hours_end) & 
    email_filtered['day_of_week'].isin(working_days))

# Drop rows sent during working hours and working days
email_filtered = email_filtered[~email_filtered['is_working_time']]

# Drop the column if no longer needed
email_filtered.drop(columns=['is_working_time'], inplace=True)

In [ ]:
#check the remaining TP users(2)
email_filtered['user'].to_csv('../data/filtered_users.csv', index=False)

#70 remaining TP cases , the action were identified as high risk to be threats that occurred outside working hours and day (9 AM - 5 PM)
#check information agianst with insider info using Verify.ipynb

In [ ]:
# Calculate the mean frequency
mean_frequency = user_frequency.mean()

# Filter users who send more emails than the mean
users_above_mean = user_frequency[user_frequency > mean_frequency]

# Filter include only these users
#filtered_email = email_filtered[email_filtered['user'].isin(users_above_mean.index)]

In [ ]:
# Predefined list of public domains
public_domains = ['gmail.com', 'yahoo.com', 'outlook.com', 'hotmail.com', 'msn.com', 'icloud.com', 'aol.com', 'live.com', 'protonmail.com', 'zoho.com', 'mail.com', 'gmx.com']

def contains_public_domain(row):
    relevant_columns = ['to', 'cc', 'bcc']
    for col in relevant_columns:
        if pd.notnull(row[col]):
            emails = row[col].split(';')
            for email in emails:
                domain = email.strip().split('@')[-1]
                if domain in public_domains:
                    return True
    return False

# create the 'is_public_domain' column
#filtered_email['is_public_domain'] = filtered_email.apply(contains_public_domain, axis=1)

# create the 'is_public_domain' column
#filtered_email['is_public_domain'] = filtered_email.apply(contains_public_domain, axis=1)
#filtered_email.loc[:, 'is_public_domain'] = filtered_email.apply(contains_public_domain, axis=1)

filtered_email = email_filtered.assign(is_public_domain=email_filtered.apply(contains_public_domain, axis=1))


In [ ]:
filtered_email['is_public_domain'].value_counts()

In [ ]:
# Separate the DataFrame into two based on the 'is_public_domain' column
public_domain_df = filtered_email[filtered_email['is_public_domain'] == True]
non_public_domain_df = filtered_email[filtered_email['is_public_domain'] == False]

In [ ]:
# Extract domains from email addresses
def extract_domains(emails):
    if pd.notnull(emails):  
            return [email.strip().split('@')[-1] for email in emails.split(';') if '@' in email]
    return []

# 'non_public_domain_df' DataFrame will get the list of domain to identify vendor_domains
#non_public_domain_df['to_domains'] = non_public_domain_df['to'].apply(extract_domains)
#non_public_domain_df['cc_domains'] = non_public_domain_df['cc'].apply(extract_domains)
#non_public_domain_df['bcc_domains'] = non_public_domain_df['bcc'].apply(extract_domains)

non_public_domain_df.loc[:, 'to_domains'] = non_public_domain_df['to'].apply(extract_domains)
non_public_domain_df.loc[:, 'cc_domains'] = non_public_domain_df['cc'].apply(extract_domains)
non_public_domain_df.loc[:, 'bcc_domains'] = non_public_domain_df['bcc'].apply(extract_domains)



In [ ]:
# List of domains with vendors from non_public_domain_df
vendor_domains = ['boeing.com', 'raytheon.com', 'hp.com', 'harris.com', 'northropgrumman.com', 'lockheedmartin.com']

def is_vendor_email(email):
    if pd.notnull(email):
        domain = email.split('@')[-1]
        return domain in vendor_domains
    return False
filtered_email['is_vendor'] = False

for index, row in filtered_email.iterrows():
    for col in ['to', 'cc', 'bcc']:
        emails = row[col].split(';') if pd.notnull(row[col]) else []
        if any(is_vendor_email(email) for email in emails):
            filtered_email.at[index, 'is_vendor'] = True
            break  

filtered_email_no_vendors = filtered_email[filtered_email['is_vendor'] == False].copy()


In [ ]:
#Data is clean
filtered_email_no_vendors

In [ ]:
filtered_email_no_vendors['user'].value_counts()

In [ ]:
public_domain_df['user'].value_counts()

In [ ]:
public_domain_df.columns

In [ ]:
contect_df = public_domain_df[['user', 'content']]

In [ ]:
contect_df.to_csv('../data/public_domain_df.csv')  #public_domain_df verify 70 TPs cases

In [ ]:
non_public_domain_df.to_csv('../data/non_public_domain_counts.csv')  #non_public_domain_df verify 6 TPs cases

In [ ]:
contect_df.nunique()

In [ ]:
# Further analysis on filtered_email_no_vendors and public_domain_df to identify the true positive
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import re

In [ ]:
def preprocess_text(text):
    if pd.isnull(text):  # Handle missing values
        return ""
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())  # Remove special characters, numbers, and convert to lowercase
    words = word_tokenize(text) # Tokenize text
    words = [word for word in words if word not in stopwords.words('english')]
    return " ".join(words)

In [ ]:
# Apply the function to the email content
contect_df.loc[contect_df.index[:1000], 'cleaned_email'] = contect_df['content'].head(3000).map(preprocess_text)

In [ ]:
contxt=contect_df.dropna()

In [ ]:
contxt.nunique()

In [ ]:
contxt.to_csv('../data/contxt.csv')  